# Quantum Circuit Picker, Related Works

**Quantum CNN**
  * Pennylane Tutorial based on PyTorch
  * Build a quantum circuit and treat it as a quantum kernel
  * The quantum kernel consists of Pennylane's RY gates and RandomLayers
  * Ref: Quanvolutional Neural Networks — PennyLane; [1904.04767] Quanvolutional Neural Networks: Powering Image Recognition with Quantum Circuits (arxiv.org)


**Quantum GAN**
  * Pennylane Tutorial based on Google Cirq and tensorflow
  * Build a generator and a discriminator that consist of Pennylane's Hadamard, RX, RY, RZ, CNOT gates
  * Ref: Quantum Generative Adversarial Networks with Cirq + TensorFlow — PennyLane


**Quantum Neural Nets**

  * Pennylane Tutorial based on Google Cirq and tensorflow
  * Build a generator and a discriminator that consist of Pennylane's Hadamard, RX, RY, RZ, CNOT gates
  * Ref: Quantum Generative Adversarial Networks with Cirq + TensorFlow — PennyLane
  
**Single Qubit Classifier**

  * Pennylane Tutorial based on Pennylane
  * Build a circuit that consists of ROT gates
  * Ref: Data-reuploading classifier — PennyLane
  
**Transfer Learning using ANN**

  * Pennylane Tutorial based on Pennylane and PyTorch
  * Build quantum layers of pennylane's Hadamard, RY, CNOT gates
  * Ref: Quantum transfer learning — PennyLane
  
**Variational Quantum Linear Solver**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's Hadamard, CZ, CNOT gates
  * Ref: Variational Quantum Linear Solver — PennyLane
  
**Coherent Variational Quantum Linear Solver**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's Hadamard, CZ, CNOT, CRY, RY gates
  * Ref: https://pennylane.ai/qml/demos/tutorial_coherent_vqls.html
  
**Multiclass Classification**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's ROT, CNOTgates
  * Ref: Multiclass margin classifier — PennyLane

**QGRNN**

  * Pennylane Tutorial based on Pennylane 
  * Build quantum circuits that consist of pennylane's ROT, CNOTgates
  * Ref: Multiclass margin classifier — PennyLane


We consider the following factors to pick quantum circuits
  * interface = "torch" # pytorch
  * simulator = "default.qubit" # cirq.simulator
  * nntype = "ann" # gan, cnn, gnn, anntf (ann for transfer learning), lstm

In [9]:
import sys
import pennylane as qml
import numpy as np

In [14]:
def qcircuit_picker(nntype, simulator, interface):
    if simulator is None: 
        simulator = "default.qubit"
    if interface is None:
        interface = "torch"
    def ann():
        Ann1QbitLayer()
        return 
    def cnn():
        CnnLayer()
        return 
    def anntf():
        AnnTfLayer()
        return 
    def gan():
        return "GAN quantum circuit picker under development..."
    def gnn():
        return "GNN quantum circuit picker under development..."
    switcher = {
        0: ann(),
        1: cnn(),
        2: anntf(),
        3: gan(),
        4: gnn()
    }
    return switcher.get(argument, "invalid neural net type")
#     switch (nntype) {
#         case "ann":
#             qlayer = Ann1QbitLayer()
#             break;
#         case "cnn":
#             qlayer = CnnLayer()
#             break;
#         case "anntf":
#             qlayer = AnnTfLayer()
#             break;
#         case "gan":
#             print('GAN quantum circuit picker under development...')
#             break;
#         case "gnn":
#             print('GNN quantum circuit picker under development...')
#             break;
#         default:
#             print('invalid neural net type')
#             break;
#     }

In [40]:
# a layer in an ANN
# This layer returns multiple measurements per wire
# For each wire, a ROT gate embedds the feature (i.e. the data sample of length 3) followed by
# another ROT gate that takes the params (i.e. the weights to optimize) as input
# The features could be the output of a preceeding layer (classical or quantum)

def AnnLayer(dev, params, features, wires):
            
        """A variational quantum circuit representing the Universal classifier.

        Args:
            params (array[float]): array of parameters
            features (array[float]): single input vector

        Returns:
            float: measurements per wire
        """
        i = 0
        ret = []
        for p in params:
            qml.Rot(*features, wires=wires[i])
            qml.Rot(*p, wires=wires[i])
            i += 1
            ret.append(qml.expval(qml.PauliZ(i)))
        return ret

In [43]:
num_wires = 5
devann = qml.device("default.qubit", wires=num_wires)
params = np.pi * np.random.random_sample((num_wires, 3))
features = np.random.uniform(0, np.pi, 3)
measurements =  AnnLayer(devann, params, features, range(num_wires))
print(measurements)

[expval(PauliZ(wires=[1])), expval(PauliZ(wires=[2])), expval(PauliZ(wires=[3])), expval(PauliZ(wires=[4])), expval(PauliZ(wires=[5]))]


In [45]:
# ref: https://github.com/PlanQK/TrainableQuantumConvolution/blob/main/Demo%20Trainable%20Quantum%20Convolution.ipynb
def CnnLayer():
    print("under development...")
#         @qml.qnode(device=self.dev, interface="torch")
#         def circuit(inputs, weights): # a quantum kernel
#             n_inputs=4
#             # Encoding of 4 classical input values
#             for j in range(n_inputs):
#                 qml.RY(inputs[j], wires=j)
#             # Random quantum circuit
#             RandomLayers(weights, wires=list(range(self.wires)), seed=seed)
            
#             # Measurement producing 4 classical output values
#             return [qml.expval(qml.PauliZ(j)) for j in range(self.out_channels)]

In [46]:
# ref example: https://pennylane.readthedocs.io/en/latest/_modules/pennylane/qnn/keras.html#KerasLayer
# ref: https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html
def AnnTfLayer():
    print("under development...")
#     nqubits = len(wires)
    
#     def H_layer(nqubits):
#     """Layer of single-qubit Hadamard gates.
#     """
#     for idx in range(nqubits):
#         qml.Hadamard(wires=idx)


#     def RY_layer(w):
#         """Layer of parametrized qubit rotations around the y axis.
#         """
#         for idx, element in enumerate(w):
#             qml.RY(element, wires=idx)


#     def entangling_layer(nqubits):
#         """Layer of CNOTs followed by another shifted layer of CNOT.
#         """
#         # In other words it should apply something like :
#         # CNOT  CNOT  CNOT  CNOT...  CNOT
#         #   CNOT  CNOT  CNOT...  CNOT
#         for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
#             qml.CNOT(wires=[i, i + 1])
#         for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
#             qml.CNOT(wires=[i, i + 1])
    
#     @qml.qnode(dev, interface="torch")
#     def quantum_net(q_input_features, q_weights_flat):
#         """
#         The variational quantum circuit.
#         """

#         # Reshape weights
#         q_weights = q_weights_flat.reshape(q_depth, n_qubits)

#         # Start from state |+> , unbiased w.r.t. |0> and |1>
#         H_layer(n_qubits)

#         # Embed features in the quantum node
#         RY_layer(q_input_features)

#         # Sequence of trainable variational layers
#         for k in range(q_depth):
#             entangling_layer(n_qubits)
#             RY_layer(q_weights[k])

#         # Expectation values in the Z basis
#         exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
#         return tuple(exp_vals)